# Getting Started with PnP Provisioning

This sample assumes that you have access to Azure AD and have an app setup on the tenant. If you don't have access to a tenant, you can [get a free developer tenant](https://developer.microsoft.com/en-us/microsoft-365/dev-program).

In [ ]:
#r "nuget: Microsoft.SharePointOnline.CSOM, 16.1.23814.12000"
#r "nuget: PnP.Framework, 1.13.0"
#r "nuget: Microsoft.Extensions.Configuration, 7.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 7.0.0"

using Microsoft.Extensions.Configuration;
using Microsoft.Extensions.Configuration.Json;
using Microsoft.SharePoint.Client;
using PnP.Framework;
using PnP.Framework.Provisioning.Providers;
using PnP.Framework.Provisioning.Providers.Xml;
using PnP.Framework.Provisioning.ObjectHandlers;
using PnP.Framework.Provisioning.Connectors;

using Model = PnP.Framework.Provisioning.Model;
using File = System.IO;

In [ ]:
// Credentials
var config = new ConfigurationBuilder()
    .AddJsonFile(Path.Combine(File.Directory.GetCurrentDirectory(),"appsettings.json"))
    .Build();
var appId = config["azureAppId"];

var certPassword = config["certificatePassword"];
var certPath = config["certificatePath"];
var tenantName = config["azureTenantName"];

## Example creating a site template based on the configuration of a existing SharePoint Site
Note: this can take between 15-30 seconds to run, depending on the amount of configuration you have changed on the SharePoint Site

In [ ]:
// SharePoint Online Site URL
string siteUrl = "https://capadevtest.sharepoint.com/sites/pnpcoresdktest";
string path = "pnpframework-sample-template.xml";

var authManager = new AuthenticationManager(appId, certPath, certPassword, tenantName);
using (var clientContext = authManager.GetContext(siteUrl))
{
    // Used to control the output of the template
    ProvisioningTemplateCreationInformation creationInformation = new ProvisioningTemplateCreationInformation(clientContext.Web);
    
    // Captures ALL the configuration information from the site
    ProvisioningTemplate template = clientContext.Web.GetProvisioningTemplate(creationInformation);

    // Output the template as XML to the file System
    XMLTemplateProvider provider = new XMLFileSystemTemplateProvider(".","");
    provider.SaveAs(template,path);

}


## Example creating a template in memory and applying a template to a SharePoint Site

Note: Sample not yet operational, work in progress...

In [ ]:
// SharePoint Online Site URL
string siteUrl = "https://capadevtest.sharepoint.com/sites/pnpcoresdktest";

var authManager = new AuthenticationManager(appId, certPath, certPassword, tenantName);
using (var clientContext = authManager.GetContext(siteUrl))
{
    // Define the site template
    ProvisioningTemplate template = new ProvisioningTemplate();
    template.Id = "ExampleTemplatePnPFramework";
    template.Version = 1.0;
    template.Scope = Model.ProvisioningTemplateScope.RootSite;
    template.BaseSiteTemplate = "SITEPAGEPUBLISHING#0";

    template.SiteFields.Add(new Model.Field() { SchemaXml = "<Field Type='Text' DisplayName='Example Field' Name='ExampleField' />" });
    template.ContentTypes.Add(new Model.ContentType() { Id = "0x0101001234567890", Name = "Example Content Type", Group = "Provisioned Group" });
    template.Lists.Add(new ListInstance() { Title = "Example List", TemplateType = 100, Url = "Lists/ExampleList", ContentTypesEnabled = true });
    
    // Serialize the site template to an XML file
    string templateFileName = "ExampleTemplatePnPFramework.xml";
    XMLTemplateProvider provider = new XMLFileSystemTemplateProvider(".", "");
    provider.SaveAs(template, templateFileName);

    

    var applyingInformation = new ProvisioningTemplateApplyingInformation();
    applyingInformation.HandlersToProcess = Handlers.All;
    
    clientContext.Web.ApplyProvisioningTemplate(template, applyingInformation);
   

    // Delete the XML file
    System.IO.File.Delete(templateFileName);
};
